# Create tables in AWS DynamoDb


In [1]:
import json
import requests
import csv
import io
import os
import pandas as pd
import numpy as np
from tqdm import tqdm



In [2]:
jsonf = requests.get("https://pubchem.ncbi.nlm.nih.gov/rest/pug/assay/aid/450/JSON")
csvf = requests.get("https://pubchem.ncbi.nlm.nih.gov/rest/pug/assay/aid/400/CSV")

In [3]:
jsonf.text

'{\n  "Fault": {\n    "Code": "PUGREST.BadRequest",\n    "Message": "Too many SIDs",\n    "Details": [\n      "Assay record retrieval is limited to 10000 SIDs"\n    ]\n  }\n}\n'

In [4]:
data = csvf.text

In [44]:
with open ("test.csv","w") as f:
    f.write(data)

In [45]:
type(data)

str

In [5]:
data

'PUBCHEM_RESULT_TAG,PUBCHEM_SID,PUBCHEM_CID,PUBCHEM_ACTIVITY_OUTCOME,PUBCHEM_ACTIVITY_SCORE,PUBCHEM_ACTIVITY_URL,PUBCHEM_ASSAYDATA_COMMENT,expt1_inhibition,expt1_IC50,expt1_enzyme_concentration,expt1_inhibitor_concentration,expt1_temperature,expt1_dtt_concentration,expt2_inhibition,expt2_IC50,expt2_enzyme_concentration,expt2_inhibitor_concentration,expt2_temperature,expt2_dtt_concentration\nRESULT_TYPE,,,,,,,FLOAT,FLOAT,FLOAT,FLOAT,FLOAT,FLOAT,FLOAT,FLOAT,FLOAT,FLOAT,FLOAT,FLOAT\nRESULT_DESCR,,,,,,,Percent inhibition of enzyme,IC50,Enzyme concentration in assay,Inhibitor concentration in assay,,DTT concentration in assay,Percent inhibition of enzyme,IC50,Enzyme concentration in assay,Inhibitor concentration in assay,,DTT concentration in assay\nRESULT_UNIT,,,,,,,PERCENT,NONE,NANOGR_PER_ML,NANOGR_PER_ML,Celcius,MILLIMOLAR,PERCENT,NONE,NANOGR_PER_ML,NANOGR_PER_ML,Celcius,MILLIMOLAR\n1,10321992,647501,Active,95,,,95,,20,50,25,1,,,,,,\n2,10321993,460748,Active,95,,,95,,20,50,25,1,,,,,,\n3,

In [47]:
data = io. StringIO(data)
df = pd.read_csv(data, sep=",")

In [117]:
FILEPATH = "../data"

class PubChemBioAssayRecordFromCsv(object):
    def __init__(self, assay_id):
        self.assay_id = assay_id
        result = requests.get("https://pubchem.ncbi.nlm.nih.gov/rest/pug/assay/aid/{0}/CSV".format(self.assay_id))
        self.record = result.text
        
    def save_csv(self):
        with open (os.path.join(FILEPATH, "pubchem_aid{0}.csv".format(self.assay_id)),"w") as f:
            f.write(self.record)
    
    def csv2df(self):
        result = io. StringIO(self.record)
        df = pd.read_csv(result, sep=",")
        st_cols = []
        exp_cols = []
        for c in list(df.columns):
            if "PUBCHEM" in c:
                st_cols += [c]
            else:
                exp_cols += [c]
                df["{}_TYPE".format(c)] = np.nan*(df.shape[0])
                df["{}_DESCR".format(c)] = np.nan*(df.shape[0])
                df["{}_UNIT".format(c)] = np.nan*(df.shape[0])
        return df, st_cols, exp_cols
    
    def get_smiles_from_cid(self):
        df = self.csv2df

In [142]:
r = PubChemBioAssayRecordFromCsv(400)

In [143]:
df, st_cols, exp_cols = r.csv2df()

In [144]:
exp_cols

['expt1_inhibition',
 'expt1_IC50',
 'expt1_enzyme_concentration',
 'expt1_inhibitor_concentration',
 'expt1_temperature',
 'expt1_dtt_concentration',
 'expt2_inhibition',
 'expt2_IC50',
 'expt2_enzyme_concentration',
 'expt2_inhibitor_concentration',
 'expt2_temperature',
 'expt2_dtt_concentration']

In [124]:
cols_to_order = []
for c in df.columns:
    if "PUBCHEM" not in c:
        cols_to_order += [c]

In [132]:
new_order = []

for c in exp_cols:
    for c2 in cols_to_order:
        if c in c2:
            new_order += [c2]
all_cols = st_cols + new_order
all_cols

df = df[all_cols]

for c in exp_cols:
    df[0][c]

In [134]:
df.head(5)

,PUBCHEM_RESULT_TAG,PUBCHEM_SID,PUBCHEM_CID,PUBCHEM_ACTIVITY_OUTCOME,PUBCHEM_ACTIVITY_SCORE,PUBCHEM_ACTIVITY_URL,PUBCHEM_ASSAYDATA_COMMENT,expt1_inhibition,expt1_inhibition_TYPE,expt1_inhibition_DESCR,...,expt2_inhibitor_concentration_DESCR,expt2_inhibitor_concentration_UNIT,expt2_temperature,expt2_temperature_TYPE,expt2_temperature_DESCR,expt2_temperature_UNIT,expt2_dtt_concentration,expt2_dtt_concentration_TYPE,expt2_dtt_concentration_DESCR,expt2_dtt_concentration_UNIT
0,RESULT_TYPE,NaN,NaN,NaN,NaN,NaN,NaN,FLOAT,NaN,NaN,...,NaN,NaN,FLOAT,NaN,NaN,NaN,FLOAT,NaN,NaN,NaN
1,RESULT_DESCR,NaN,NaN,NaN,NaN,NaN,NaN,Percent inhibition of enzyme,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,DTT concentration in assay,NaN,NaN,NaN
2,RESULT_UNIT,NaN,NaN,NaN,NaN,NaN,NaN,PERCENT,NaN,NaN,...,NaN,NaN,Celcius,NaN,NaN,NaN,MILLIMOLAR,NaN,NaN,NaN
3,1,10321992.0,647501.0,Active,95.0,NaN,NaN,95,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2,10321993.0,460748.0,Active,95.0,NaN,NaN,95,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
import json
import requests
from tqdm import tqdm

HEADER = "PC_AssaySubmit"
PUBCHEM_PREFIX = "PUBCHEM"

COMPOUND_QUERY_BATCH_SIZE = 100


class PubChemBioAssayRecordFromJson(object):

    def __init__(self, assay_id=None, assay_json_file=None, batch_size=None):
        if assay_json_file is not None:
            with open(assay_json_name, "r") as f:
                self.record = json.load(f)
        elif assay_id is not None:
            result = requests.get("https://pubchem.ncbi.nlm.nih.gov/rest/pug/assay/aid/{0}/JSON".format(assay_id))
            self.record = json.loads(result.text)
        else:
            raise Exception
        self.header = HEADER
        self.prefix = PUBCHEM_PREFIX
        if batch_size is None:
            self.batch_size = COMPOUND_QUERY_BATCH_SIZE
        else:
            self.batch_size = batch_size

    def get_id(self, record):
        return record[HEADER]["assay"]["descr"]["aid"]["id"]

    def get_sids(self, record):
        sids = []
        for d in record[HEADER]["data"]:
            sids += [d["sid"]]
        return sids

    def chunker(self, seq, size):
        return (seq[pos:pos + size] for pos in range(0, len(seq), size))

    def get_cids_from_sids(self, sids):
        s = ",".join([str(sid) for sid in sids])
        url = "https://pubchem.ncbi.nlm.nih.gov/rest/pug/substance/sid/{0}/cids/JSON".format(s)
        r = requests.get(url)
        data = json.loads(r.text)
        if "InformationList" not in data:
            return {}
        data = data["InformationList"]["Information"]
        sid2cids = {}
        for d in data:
            if "CID" not in d:
                continue
            sid2cids[d["SID"]] = d["CID"]
        return sid2cids

    def get_smiles_from_cids(self, cids):
        s = ",".join([str(cid) for cid in list(set(cids))])
        url = "https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/cid/{0}/property/CanonicalSmiles,IsomericSmiles/JSON".format(s)
        print(url)
        r = requests.get(url)
        data = json.loads(r.text)["PropertyTable"]["Properties"]
        cid2smiles = {}
        for d in data:
            if "CanonicalSMILES" in d:
                cid2smiles[d["CID"]] = d["CanonicalSMILES"]
            elif "IsomericSMILES" in d:
                cid2smiles[d["CID"]] = d["IsomericSMILES"]
            else:
                continue
        return cid2smiles

    def get_substances(self, sids):
        sid2cids = self.get_cids_from_sids(sids)
        all_cids = list(set([x for _,v in sid2cids.items() for x in v]))
        if not all_cids:
            return [(sid, None, None) for sid in sids]
        cid2smiles = self.get_smiles_from_cids(all_cids)
        compounds = []
        for sid in sids:
            if sid not in sid2cids:
                compounds += [(sid, None, None)]
            else:
                for cid in sid2cids[sid]:
                    found = False
                    if not found and cid in cid2smiles:
                        compounds += [(sid, cid, cid2smiles[cid])]
                        found = True
                if not found:
                    compounds += [(sid, None, None)]
        return compounds

    def get_compounds_from_sids(self, sids):
        done_sids = set()
        compounds = {}
        todo_sids = set(sids)
        i = 0
        while len(todo_sids) > 0:
            print("Attempt {0}".format(i))
            sids_ = list(todo_sids)
            for j, chunk in enumerate(self.chunker(sids_, self.batch_size)):
                print("Chunk {0}. Done: {1}".format(j, len(compounds)))
                for subs in tqdm(self.get_substances(chunk)):
                    compounds[subs[0]] = (subs[1], subs[2])
                    done_sids.update([subs[0]])
            todo_sids = todo_sids.difference(done_sids)
            i += 1
        compounds = [compounds[sid] for sid in sids]
        return compounds

    def get_description(self, record):
        return record[HEADER]["assay"]["descr"]

    def get_data(self, record):
        return record[HEADER]["data"]

    def get_data_with_compounds(self, record):
        sids = self.get_sids(record) 
        compounds = self.get_compounds_from_sids(sids)
        data = self.get_data(record)
        for i in range(len(data)):
            cid = compounds[i][0]
            smiles = compounds[i][1]
            data[i]["cid"] = cid
            data[i]["smiles"] = smiles
        return data

    def get(self):
        print(self.record)
        result = {
            "assay_id": self.get_id(self.record),
            "description": self.get_description(self.record),
            "data": self.get_data_with_compounds(self.record)
        }
        return {"AssayId": "{0}{1}".format(PUBCHEM_PREFIX, result["assay_id"]), "Description": result["description"], "Data": result["data"]}

In [16]:
r = PubChemBioAssayRecordFromJson(assay_id=400, batch_size=500)

In [17]:
data = r.get()

{'PC_AssaySubmit': {'assay': {'descr': {'aid': {'id': 400, 'version': 1}, 'aid_source': {'db': {'name': 'SGCOxCompounds', 'source_id': {'str': 'Activity Assay, Human PTPN5'}}}, 'name': 'Activity Assay, Human PTPN5', 'description': ['STEP, a striatal-enriched protein tyrosine phosphatase, is preferentially expressed in neurons of the basal ganglia, hippocampus, cortex and related structures. Alternative splicing produces various STEP family members, and both cytosolic (STEP 46) and membrane-associated (STEP 61) variants exist. STEP and its non-neuronal homologs like He-PTP have been implicated in the regulation of ERK activity. Both splice products STEP 61 and STEP 46 are phosphorylated in a common kinase-interacting domain (KIM) that binds to ERK2. The kinase PKA phosphorylates STEP 46 at Ser49 (equivalent to Ser221 in STEP 61) which decreases the activity of STEP by reducing its affinity for its substrates. Dephosphorylated STEP binds and inactivates ERK through dephosphorylation of t

https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/cid/54692867,135403652,135403651,135403653,135403655,654089,4373397,4373399,4373401,6419740,6419742,6419743,6419744,6165025,4352420,805156,3136298,6420095,5335985,823994,738749,1607,460748,647501,1354702,5206350,545872,5756371,5235,893428,745336,6420090,6420091,1023/property/CanonicalSmiles,IsomericSmiles/JSON


100%|██████████████████████████████████████████████████████████████████████████████████████████| 36/36 [00:00<?, ?it/s]


In [31]:
d = data["Data"]

In [35]:
d[1]

{'sid': 10321993,
 'version': 0,
 'outcome': 2,
 'rank': 95,
 'data': [{'tid': 1, 'value': {'fval': 95}},
  {'tid': 3, 'value': {'fval': 20}},
  {'tid': 4, 'value': {'fval': 50}},
  {'tid': 5, 'value': {'fval': 25}},
  {'tid': 6, 'value': {'fval': 1}}],
 'cid': 460748,
 'smiles': 'CN1C2=NC(=O)N(C(=O)C2=NC(=N1)C3=CC=CC=C3)C'}

In [36]:
result = requests.get("https://pubchem.ncbi.nlm.nih.gov/rest/pug/assay/aid/400/JSON")
rec = json.loads(result.text)
data_all = rec["PC_AssaySubmit"]["data"]

In [37]:
data_all

[{'sid': 10321992,
  'version': 0,
  'outcome': 2,
  'rank': 95,
  'data': [{'tid': 1, 'value': {'fval': 95}},
   {'tid': 3, 'value': {'fval': 20}},
   {'tid': 4, 'value': {'fval': 50}},
   {'tid': 5, 'value': {'fval': 25}},
   {'tid': 6, 'value': {'fval': 1}}]},
 {'sid': 10321993,
  'version': 0,
  'outcome': 2,
  'rank': 95,
  'data': [{'tid': 1, 'value': {'fval': 95}},
   {'tid': 3, 'value': {'fval': 20}},
   {'tid': 4, 'value': {'fval': 50}},
   {'tid': 5, 'value': {'fval': 25}},
   {'tid': 6, 'value': {'fval': 1}}]},
 {'sid': 10321994,
  'version': 0,
  'outcome': 2,
  'rank': 89,
  'data': [{'tid': 1, 'value': {'fval': 89}},
   {'tid': 3, 'value': {'fval': 20}},
   {'tid': 4, 'value': {'fval': 200}},
   {'tid': 5, 'value': {'fval': 25}},
   {'tid': 6, 'value': {'fval': 1}},
   {'tid': 7, 'value': {'fval': 46}},
   {'tid': 9, 'value': {'fval': 87}},
   {'tid': 10, 'value': {'fval': 50}},
   {'tid': 11, 'value': {'fval': 25}},
   {'tid': 12, 'value': {'fval': 5}}]},
 {'sid': 103219

In [267]:
import json
import requests

PUBCHEM_PREFIX = "PUBCHEM"


class PubChemBioAssayRecordFromJson(object):
    
    def __init__(self, assay_id=None, assay_json_file=None, batch_size=10000):
        self.batch_size=batch_size
        if assay_json_file is not None:
            with open(assay_json_name, "r") as f:
                self.record = json.load(f)
        elif assay_id is not None:
            self.url = "https://pubchem.ncbi.nlm.nih.gov/rest/pug/assay/aid/{}".format(assay_id)
            r = requests.get("{0}/sids/json".format(self.url))
            self.record = json.loads(r.text)
    
    def _chunker(self, seq, size):
        return (seq[pos:pos + size] for pos in range(0, len(seq), size))

    def _get_id(self, record):
        return record["InformationList"]["Information"][0]["AID"]
    
    def _get_sids(self, record):
        return record["InformationList"]["Information"][0]["SID"]
    
    def _get_description(self, record):
        req = requests.get("{}/description/json".format(self.url))
        record = json.loads(req.text)
        return record["PC_AssayContainer"][0]["assay"]["descr"]

    def _get_data(self, record):
        data = []
        for chunk in self._chunker(self._get_sids(record), self.batch_size):
            s = ",".join([str(sid) for sid in chunk])
            r = requests.post("{0}/json".format(self.url), data={'sid': s})
            result = json.loads(r.text)["PC_AssaySubmit"]["data"]
            data.append(result)
        data = [d for chunk in data for d in chunk]
        return data

    def _get_cids_from_sids(self, sids):
        sid2cids = {}
        for chunk in self._chunker(sids, self.batch_size):
            s = ",".join([str(sid) for sid in chunk])
            url = "https://pubchem.ncbi.nlm.nih.gov/rest/pug/substance/sid/cids/json"
            r = requests.post(url, data={'sid': s})
            result = json.loads(r.text)
            if "InformationList" not in result:
                return {}
            result = result["InformationList"]["Information"]
            for res in result:
                if "CID" not in res:
                    continue
                sid2cids[res["SID"]] = res["CID"]
        return sid2cids
    

    def _get_smiles_from_cids(self, cids):
        cid2smiles = {}
        for chunk in self._chunker(cids, self.batch_size):
            s = ",".join([str(cid) for cid in list(set(cids))])
            url = "https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/cid/property/CanonicalSmiles,IsomericSmiles/JSON".format(s)
            r = requests.post(url, data={"cid": s})
            result = json.loads(r.text)["PropertyTable"]["Properties"]
            for res in result:
                if "CanonicalSMILES" in res:
                    cid2smiles[res["CID"]] = res["CanonicalSMILES"]
                elif "IsomericSMILES" in res:
                    cid2smiles[res["CID"]] = res["IsomericSMILES"]
                else:
                    continue
        return cid2smiles
    
    def _get_substances(self, sids):
        sid2cids = self._get_cids_from_sids(sids)
        all_cids = list(set([c for cid in list(sid2cids.values()) for c in cid]))
        if not all_cids:
            return [(sid, None, None) for sid in sids]
        cid2smiles = self._get_smiles_from_cids(all_cids)
        compounds = []
        for sid in sids:
            if sid not in sid2cids:
                compounds += [(sid, None, None)]
            else:
                for cid in sid2cids[sid]:
                    found = False
                    if not found and cid in cid2smiles:
                        compounds += [(sid, cid, cid2smiles[cid])]
                        found = True
                if not found:
                    compounds += [(sid, None, None)]
        return compounds
    
    def _get_compounds_from_sids(self, sids):
        compounds = {}
        for subs in self._get_substances(sids):
            compounds[subs[0]] = (subs[1], subs[2])
        compounds = [compounds[sid] for sid in sids]
        return compounds
    
    
    def _get_data_with_compounds(self, record):
        sids = self._get_sids(record) 
        compounds = self._get_substances(sids)
        data = self._get_data(record)
        for i in range(len(data)):
            sid = data[i]["sid"]
            for c in compounds:
                if c[0] == sid:
                    cid = c[1]
                    smiles = c[2]
                    data[i]["cid"] = cid
                    data[i]["smiles"] = smiles
        return data
    
    def get(self):
        result = {
            "assay_id": self._get_id(self.record),
            "description": self._get_description(self.record),
            "data": self._get_data_with_compounds(self.record)
        }
        return {"AssayId": "{0}{1}".format(PUBCHEM_PREFIX, result["assay_id"]), 
                "Description": result["description"], 
                "Data": result["data"]}

In [268]:
r = PubChemBioAssayRecordFromJson(assay_id=450)
data = r.get()

In [269]:
data

{'AssayId': 'PUBCHEM450',
 'Description': {'aid': {'id': 450, 'version': 1},
  'aid_source': {'db': {'name': 'NCGC', 'source_id': {'str': 'grgfp'}}},
  'name': 'GR-GFP Redistribution',
  'description': ['NIH Molecular Libraries Screening Centers Network [MLSCN]',
   'NIH Chemical Genomics Center [NCGC]',
   '',
   'NCGC Assay Overview:',
   'The glucocorticoid receptor (GR) Redistribution assay (BioImage) enables the visualization of GR cytoplasmic to nuclear translocation by the use of a GR-GFP fusion.  GR is normally cytosolic, however ligands such as dexamethasone, cause nuclear translocation where the protein binds to response elements and interacts with various co-factors to modulate transcription.  Because both functional agonists and antagonists can induce nuclear translocation, this assay can detect ligands regardless of their effects on gene expression. GR-GFP expressing U20S cells were used here. Dexamethasone was the positive control for translocation and data normalization.

In [ ]:
file_name = "../7001.concise.json"        

In [ ]:
import json
from pubchempy import get_substances


HEADER = "PC_AssaySubmit"
PUBCHEM_PREFIX = "PUBCHEM"


class PubChemBioAssayRecordFromJson(object):
    
    def __init__(self, json_file):
        with open(file_name, "r") as f:
            self.record = json.load(f)
        
    def get_id(self, record):
        return record[HEADER]["assay"]["descr"]["aid"]["id"]

    def get_sids(self, record):
        sids = []
        for d in record[HEADER]["data"]:
            sids += [d["sid"]]
        return sids    

    def get_compounds_from_sids(self, sids):
        compounds = []
        for subs in get_substances(sids):
            compounds += [(subs.standardized_cid, subs.standardized_compound.canonical_smiles)]
        return compounds

    def get_description(self, record):
        return record[HEADER]["assay"]["descr"]

    def get_data(self, record):
        return record[HEADER]["data"]

    def get_data_with_compounds(self, record):
        sids = self.get_sids(record)
        compounds = self.get_compounds_from_sids(sids)
        data = self.get_data(record)
        for i in range(len(data)):
            sid = data[i]["sid"]
            
            data[i]["cid"] = compounds[i][0]
            data[i]["smiles"] = compounds[i][1]
        return data

    def get(self):
        result = {
            "assay_id": self.get_id(self.record),
            "description": self.get_description(self.record),
            "data": self.get_data_with_compounds(self.record)
        }
        return {KEYNAME: "{0}{1}".format(PUBCHEM_PREFIX, result["assay_id"]), "Description": result["description"], "Data": result["data"]}
        
        
getter = PubChemBioAssayRecordFromJson("../7001.concise.json")

In [ ]:
data = getter.get()

In [ ]:
https://ftp.ncbi.nlm.nih.gov/pubchem/Bioassay/Concise/JSON/0000001_0001000.zip

In [ ]:
import requests

result = requests.get("https://pubchem.ncbi.nlm.nih.gov/rest/pug/assay/aid/7001/JSON")
json.loads(result.text)